# Fitting Description

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


In [ ]:
!pip install -r requirements.txt

In [1]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


### SVC

In [272]:
import logging
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from definitions import DATA_PATH, SUB_PATH, TRAIN_IMAGES
from src.predict import prediction
from airotica.airotica import detectron


%matplotlib inline
%load_ext autoreload
%autoreload 2

log = logging.getLogger(__name__)

In [278]:
train = pd.read_csv(DATA_PATH / 'train.csv')
train_url_only = pd.read_csv(DATA_PATH / 'train_url_only.csv')
train_url_loaded_images = pd.read_csv(DATA_PATH / 'train_loaded_images.csv')
test = pd.read_csv(DATA_PATH / 'test.csv')
sample_submission = pd.read_csv(DATA_PATH / 'sample_submission.csv')

train_labels = train.typology.unique()
typology_to_label = dict(zip(sorted(train_labels), range(len(train_labels))))
label_to_typology = {v:k for k, v in typology_to_label.items()}


MAPPER = {0: 0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 
          7:7, 8:8, 9:9, 10:10, 11:12, 12:13, 13:14,
          -1: -1
          }

In [ ]:
images_prediction = detectron(config_name='config5.yml', 
                              save_path='./sub/predictv3.csv',
                              model_path='./EXPERIMENTS/full_res/model_0001_0.857586.pth',
                              model_type='resnext101_32x8d',
                              save_result=False,
                              return_result=True,
                              use_description=False
                              )
description_prediction = prediction('predict.yaml')

In [281]:
test = test.drop('typology', axis=1)
test['typology_desc'] = -1
test['typology_images'] = -1
test.loc[description_prediction.index, 'typology_desc'] = description_prediction.values
d = images_prediction[images_prediction.sign_0 != -1]
test.loc[d.index, 'typology_images'] = d.sign_0.map(MAPPER).map(label_to_typology)

In [298]:
import os
from os import listdir
train_images = listdir(TRAIN_IMAGES)
guid_train_images = [f.split('.')[0] for f in train_images if os.path.getsize(f"{str(TRAIN_IMAGES / f)}") != 0]

In [299]:
dummy = test[test.guid.isin(guid_train_images)]
test_only_description = test[~(test.guid.isin(guid_train_images))]
test_only_images = dummy[dummy.description.isna()]
test_images_and_description = dummy[~(dummy.description.isna())]
print('without image or description:', len(test_only_description[test_only_description.description.isna()]))
print('only description:', test_only_description.shape[0])
print('only images:', test_only_images.shape[0])
print('images and description:', test_images_and_description.shape[0])

without image or description: 0
only description: 106
only images: 547
images and description: 570


In [300]:
true_labels = test_images_and_description[test_images_and_description.typology_desc == test_images_and_description.typology_images]
fake_labels = test_images_and_description[test_images_and_description.typology_desc != test_images_and_description.typology_images]

In [407]:
cat = ['фотографии и негативы', 'графика', 'живопись']
tps = fake_labels[fake_labels.typology_images.isin(cat)]
tps2 = fake_labels[~(fake_labels.typology_images.isin(cat))]
sample_submission.loc[tps.index, 'typology'] = tps.typology_images
sample_submission.loc[tps2.index, 'typology'] = tps2.typology_desc
sample_submission.loc[test_only_images.index, 'typology'] = test_only_images.typology_images
sample_submission.loc[test_only_description.index, 'typology'] = test_only_description.typology_desc

In [ ]:
sample_submission.to_csv('./sub/bblbl4.csv', index=False)

In [ ]:
for i, row in fake_labels[fake_labels.typology_images == 'графика'].iterrows():
    filename = row['guid']
    desc = row['typology_desc']
    im = row['typology_images']
    img = read_image(str(TRAIN_IMAGES / f'{filename}.jpg'))
    plt.imshow(img, label='1')
    plt.title(row['description'] + '\n' + f'{desc} ------ {im}')
    plt.legend()
    plt.show()


In [ ]:
from airotica.utils import read_image
import matplotlib.pyplot as plt


for i, row in fake_labels.iterrows():
    filename = row['guid']
    desc = row['typology_desc']
    im = row['typology_images']
    img = read_image(str(TRAIN_IMAGES / f'{filename}.jpg'))
    plt.imshow(img, label='1')
    plt.title(row['description'] + '\n' + f'{desc} ------ {im}')
    plt.legend()
    plt.show()

In [342]:
def apply_new_desc(text):
    text = text + " - это"
    input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
    out = model.generate(input_ids.cuda(), max_length=50)
    generated_text = list(map(tokenizer.decode, out))[0]
    return generated_text

In [ ]:
fake_labels['description_generated'] = fake_labels['description'].apply(apply_new_desc)

In [345]:
s = pd.DataFrame()
s['description'] = fake_labels['description_generated']

In [346]:
pred_gen = prediction('predict.yaml', test=s)

tokenizing:   0%|          | 0/120 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
tokenizing: 100%|██████████| 120/120 [00:00<00:00, 3860.50it/s]


1/1 [==============================] - 1s 610ms/step


tokenizing: 100%|██████████| 120/120 [00:00<00:00, 3702.63it/s]


1/1 [==============================] - 1s 602ms/step


In [406]:
pred_gen

1                            прочие
14                           прочие
28                           прочие
29              предметы археологии
38      предметы печатной продукции
                   ...             
1168    предметы печатной продукции
1169                        графика
1184           предметы нумизматики
1189                        графика
1198                         прочие
Length: 120, dtype: object

In [348]:
fake_labels['typology_gen'] = pred_gen

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [338]:
for i, row in fake_labels.iterrows():
    text = row['description'] + " - это"
    input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
    out = model.generate(input_ids.cuda(), max_length=50)
    generated_text = list(map(tokenizer.decode, out))[0]
    print(row['description'])
    print(generated_text)
    print('--', row['typology_desc'])
    print('\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Образец волос. Удмурты
Образец волос. Удмурты - это не только красивые, но и очень умные люди.

В Удмуртии есть свой национальный праздник - День оленевода.

В этом году он выпадает на 5 мая.

В этом
-- предметы прикладного искусства, быта и этнографии




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Диафильм для детей «Конек – Горбунок. Часть 4»
Диафильм для детей «Конек – Горбунок. Часть 4» - это увлекательная история о том, как мальчик Конек и его друзья попали в волшебную страну, где живут добрые и злые гномы.
В этой сказке рассказывается
-- прочие




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Открытка немаркированная художественная двойная (сгиб слева) "С Новым годом!". Рисунок художника А. Семенова
Открытка немаркированная художественная двойная (сгиб слева) "С Новым годом!". Рисунок художника А. Семенова - это не просто поздравление, а целая история. В ней есть и любовь, и боль, и радость,
-- графика




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Медаль памятная "XXII Олимпиада. Москва" Ананьина А.Е. В футляре.
Медаль памятная "XXII Олимпиада. Москва" Ананьина А.Е. В футляре. - это медаль, которая вручается за выдающиеся спортивные достижения. Она имеет форму круга диаметром 32 мм. На лицевой стороне медали
-- предметы нумизматики




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 62, but ``max_length`` is set to 50.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


Плакат. «Под знаменем Ленина- вперед к Победе!».
Плакат. «Под знаменем Ленина- вперед к Победе!». - это не просто лозунг, это гимн, который должен быть в каждом доме.

В этом году в рамках празднования 70-летия Победы в Великой Отечественной войне, в городе
-- графика


Грампластинка. 1 ст.: "Голубка моя". Оркестр "Гармония" под упр. В.Варшавского. 2 ст.:"Маршъ Черняева". оркестр "Гармония" под упр.В.Варшавского.
Грампластинка. 1 ст.: "Голубка моя". Оркестр "Гармония" под упр. В.Варшавского. 2 ст.:"Маршъ Черняева". оркестр "Гармония" под упр.В.Варшавского. - это было
-- прочие




In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()


text = """Божок в одежде."""
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
out = model.generate(input_ids.cuda(), max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)